In [1]:
import os
import numpy as np
import tensorflow as tf
import cv2

MODEL_PATH = "models/emotion_model.h5"
IMG_SIZE = 96
EMOTION_LABELS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

def detect_emotion_realtime(model_path=MODEL_PATH):
    if not os.path.exists(model_path):
        print(f" Model not found at {model_path}. Please train it first.")
        return

    model = tf.keras.models.load_model(model_path)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    print("Starting Real-Time Emotion Detection... (Press 'q' to quit)")

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
            face_norm = face_resized.astype("float32") / 255.0
            preds = model.predict(np.expand_dims(face_norm, axis=0), verbose=0)
            label_idx = np.argmax(preds)
            label = EMOTION_LABELS[label_idx]
            conf = np.max(preds)

            color = (0, 255, 0) if label == "Happy" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label} ({conf*100:.1f}%)", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

        cv2.imshow("Emotion Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(" Real-time detection stopped.")

if __name__ == "__main__":
    detect_emotion_realtime()

Starting Real-Time Emotion Detection... (Press 'q' to quit)
 Real-time detection stopped.
